`ApJdataFrames` Pecault2013
---
`Title`: INTRINSIC COLORS, TEMPERATURES, AND BOLOMETRIC CORRECTIONS OF PRE-MAIN-SEQUENCE STARS  
`Authors`: Mark J. Pecaut and Eric E. Mamajek

Data is from this paper:  
http://iopscience.iop.org/0067-0049/208/1/9/article

In [ ]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_context("notebook", font_scale=1.5)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from astropy.io import ascii

## Table 1
### Spectral Types and Optical/Near-IR Photometry for Young, Nearby, Moving Group Members

In [ ]:
#! curl http://iopscience.iop.org/0067-0049/208/1/9/suppdata/apjs480616t1_mrt.txt > ../data/Pecaut2013/tbl1_raw.txt

In [ ]:
tbl1 = ascii.read("../data/Pecaut2013/tbl1_raw.txt")

In [ ]:
tbl1_df = tbl1.to_pandas()

In [ ]:
import gully_custom

#### Take the subset of the M-stars, and convert the spectral types to a number, and compute I-J as a function of Spectral Type.

In [ ]:
tbl1_df["Ic"] = tbl1_df["Vmag"] - tbl1_df["V-Ic"]
tbl1_df["Ic-J"] = tbl1_df["Ic"] - tbl1_df["Jmag"]

In [ ]:
tbl1_sub = tbl1_df[tbl1_df.SpType.str.contains("M")]
tbl1_sub["SpT_clean"] = tbl1_sub.SpType.str.strip("VIe()+-}{")
tbl1_sub.SpT_clean[49] = "M8"
tbl1_sub["SpT_num"] = gully_custom.specType(tbl1_sub.SpT_clean)

In [ ]:
tbl1_sub.to_csv("../data/Pecaut2013/tbl1_M_subset.txt", index=False)

## Table 6 - Intrinsic Colors of 5–30 Myr Old Stars and Adopted Teff, Bolometric Correction Values

In [ ]:
#tbl6 = ascii.read("http://iopscience.iop.org/0067-0049/208/1/9/suppdata/apjs480616t6_mrt.txt")
tbl6 = ascii.read("../data/Pecaut2013/tbl6_raw.txt")

In [ ]:
tbl6.columns

In [ ]:
tbl6[["SpType", "Teff", "BCJ"]]

Boo! Their $BC_J$ does not go past M5!

In [ ]:
tbl6_df = tbl6.to_pandas()#

In [ ]:
tbl6_df.to_csv("../data/Pecaut2013/tbl6.csv")

In [ ]:
from gully_custom import specType

In [ ]:
tbl6_df['SpT'] = specType(tbl6_df.SpType)

In [ ]:
tbl6_df.head()

From [this website](http://www.astro.caltech.edu/~jmc/2mass/v3/transformations/).

$(J-H)_{2MASS}= (1.156 \pm 0.015) (J-H)_{MKO}+(-0.038 \pm 0.006)$

$(J-H)_{MKO} = [(J-H)_{2MASS} + (0.038 \pm 0.006)] * \frac{1}{(1.156 \pm 0.015) }$

In [ ]:
jh_mko = (tbl6_df["J-H"].values+0.038)*1.0/1.156

In [ ]:
tbl6_df["J-H_MKO"] = jh_mko
tbl6_df.to_csv("../data/Pecaut2013/tbl6.csv", index=False)

In [ ]:
X = tbl6_df.SpT.values[20:]
y = tbl6_df["J-H"].values[20:]
z = np.polyfit(X, y, 10)
p = np.poly1d(z)
y_pred = p(X)


plt.plot(tbl6_df.SpT, tbl6_df["J-H"], 's', label='2MASS')
plt.plot(tbl6_df.SpT, tbl6_df["J-H_MKO"], 'o', label='MKO')
plt.plot(X, y_pred, 'k:', label='fit')
plt.legend()
plt.xlim(0, 10)
plt.ylim(0, 1)

## Table 10 - T_eff, Bolometric Correction, and Bolometric Magnitude Polynomial Coefficients for 5-30 Myr Old Stars

In [ ]:
import pandas as pd

In [ ]:
tbl10 = pd.read_csv("http://iopscience.iop.org/0067-0049/208/1/9/suppdata/apjs480616t10_ascii.txt", skip_footer=1,
                    skiprows=[0,1,2], header=0, sep='\t', na_values='\ldots')
del tbl10["Unnamed: 11"]
tbl10

The coefficients are:  
$Y = a_0 + a_1X$

In [ ]:
a_0 = np.float(tbl10.get_value(6, "a_0")[0:8])*10**(0.0)
a_1 = np.float(tbl10.get_value(6, "a_1")[0:9])*10**(-4.0)
print a_0, a_1

In [ ]:
X = tbl6["Teff"]
Y = tbl6["BCJ"]
p = np.poly1d([a_1, a_0]) # Input: "The polynomial's coefficients, in decreasing powers"
print p
Y_fit = p(X)
print Y_fit.shape, X.shape

In [ ]:
plt.plot(X, Y, '.', label='Table 6')
plt.plot(X, Y_fit, label='Fit from Table 10')
plt.title("Pecault and Mamajek 2013")
plt.legend(loc='best')

*Script finished.*